In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import words
import csv,random,pickle

In [ ]:
#parsing data

english_word_list = words.words()
random.shuffle(english_word_list)
english_word_list = list(map(lambda x:[x,0],english_word_list))

company_names = pd.read_csv("companies_sorted.csv",usecols = ["name"]).values.tolist()
company_names = list(map(lambda x:[x[0],1],company_names))[:250000]

company_names.extend(english_word_list)

inputs_labels = company_names
random.shuffle(inputs_labels)

for line in inputs_labels:
    try:
        line[0].lower()
    except:
        inputs_labels.remove(line)

    if len(line[0]) == 0:
        inputs_labels.remove(line)

In [ ]:
X = list(map(lambda x:x[0],inputs_labels))
Y = list(map(lambda x:x[1],inputs_labels))

In [ ]:
#*********************TOKENISER*********************

tkn = Tokenizer()
tkn.fit_on_texts(X)
total_chars = len(tkn.word_index) + 1

tokenised_list = tkn.texts_to_sequences(X)

maxSequenceLen = max([len(x) for x in tokenised_list])
inputSequences = np.array(pad_sequences(tokenised_list,maxlen = maxSequenceLen,padding = "pre"))

In [129]:
#******************SAVE*TOKENISER********************

with open("single_token_vendor_tokenizer.pickle","wb") as file:
    pickle.dump(tkn,file,protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
#*********************MODEL*********************

model = tf.keras.Sequential()

model.add(Embedding(total_chars,50,input_length = maxSequenceLen))
model.add(LSTM(64))
model.add(Dense(256,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(loss = "mse",optimizer = "adam")

In [ ]:
tf.config.run_functions_eagerly(True)

model.fit(inputSequences,np.array(Y),epochs = 2,verbose = 1)

In [ ]:
model.fit(inputSequences,np.array(Y),epochs = 4,verbose = 1)

In [ ]:
tokenised_sequence = tkn.texts_to_sequences(["with thanks"])

test_sequence = np.array(pad_sequences(tokenised_sequence,maxlen = maxSequenceLen,padding = "pre"))

model.predict(test_sequence)

In [ ]:
model.save("single_token_vendor_model")